In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, UpSampling2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, precision_recall_curve, accuracy_score, zero_one_loss, balanced_accuracy_score, average_precision_score
from inspect import signature
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
import os
import pickle
import sys
import logging
from tensorflow.keras import layers
from tensorflow.keras import Input
import seaborn as sns
from tensorflow.keras import backend
from sklearn.metrics import classification_report

from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE

### Validando Red VGG16 1000, 800, 400 HGT

In [4]:
model = load_model("/opt/datos/entrenamientos/larraondo/vgg16_val_0_2_6.h5")

In [6]:
x_test = np.load("/opt/datos/dataset/larraondo/x_test-0_2_6_validation.npy")
y_test = np.load("/opt/datos/dataset/larraondo/y_test-0_2_6_validation.npy")

print(x_test.shape)
print(y_test.shape)

(10805, 80, 120, 3)
(10805, 80, 120, 1)


In [7]:
for i in range(10805):
    print("-----------------------------------------------------------------------")
    rain_pred = model.predict(x_test[i:i+1,:])
    rain = y_test[i:i+1,:]
    
    #TOTAL DE PRECIPITACION
    print("Total pred", np.sum(rain_pred))
    print("Total", np.sum(rain))
    
    #MEDIA PRECIPITACION
    print("Mean pred", np.sum(rain_pred) / float(rain.shape[1] * rain.shape[2]))
    print("Mean", np.sum(rain) / float(rain.shape[1] * rain.shape[2])) 
    
    # ERRORES
    mae = np.sum(rain_pred - rain)
    print("E", mae) 
    mae = np.sum(np.absolute(rain_pred - rain))
    print("AE", mae) 
    mae /= float(rain.shape[1] * rain.shape[2])
    print("MAE", mae) 
    print("-----------------------------------------------------------------------")

Total pred 3469.351
Total 3776.3728
Mean pred 0.36139073689778645
Mean 0.3933721669514974
E -307.02167
AE 2474.2087
MAE 0.25773007710774737
Total pred 6341.385
Total 6937.786
Mean pred 0.6605609130859375
Mean 0.722686055501302
E -596.40155
AE 2807.708
MAE 0.2924695841471354
Total pred 4523.8223
Total 5084.4873
Mean pred 0.47123148600260417
Mean 0.5296340942382812
E -560.6652
AE 2676.4395
MAE 0.2787957763671875
Total pred 5242.219
Total 5596.968
Mean pred 0.5460645039876302
Mean 0.5830174763997396
E -354.7485
AE 2952.3755
MAE 0.30753911336263023
Total pred 3534.582
Total 3874.6558
Mean pred 0.36818562825520834
Mean 0.40360997517903646
E -340.0738
AE 2831.7866
MAE 0.29497777303059897
Total pred 2762.6875
Total 3420.392
Mean pred 0.28777994791666667
Mean 0.3562908426920573
E -657.70465
AE 2417.902
MAE 0.2518648020426432
Total pred 4653.068
Total 5278.6416
Mean pred 0.48469456990559895
Mean 0.5498585001627604
E -625.57367
AE 3216.3423
MAE 0.33503565470377605
Total pred 3629.718
Total 3975.

Total pred 3173.5522
Total 3531.0308
Mean pred 0.33057835896809895
Mean 0.3678157043457031
E -357.47842
AE 2273.8562
MAE 0.23686002095540365
Total pred 4205.2104
Total 4909.8013
Mean pred 0.4380427551269531
Mean 0.5114376322428386
E -704.591
AE 2828.6047
MAE 0.29464632670084634
Total pred 4486.608
Total 4867.791
Mean pred 0.46735499064127606
Mean 0.5070615641276042
E -381.18326
AE 3209.6572
MAE 0.3343392944335937
Total pred 2841.3262
Total 2734.4246
Mean pred 0.29597147623697917
Mean 0.28483589172363283
E 106.90166
AE 2051.2812
MAE 0.21367513020833334
Total pred 2294.1172
Total 2362.1912
Mean pred 0.23897054036458334
Mean 0.24606157938639323
E -68.07408
AE 1750.7711
MAE 0.18237199147542318
Total pred 3601.9011
Total 3730.6328
Mean pred 0.37519803365071613
Mean 0.38860758463541667
E -128.73145
AE 1986.2625
MAE 0.20690233866373697
Total pred 4239.706
Total 5109.001
Mean pred 0.44163604736328127
Mean 0.5321876017252604
E -869.29553
AE 3095.4172
MAE 0.3224392954508464
Total pred 6099.968
T

Total pred 4035.1245
Total 4397.3887
Mean pred 0.42032546997070314
Mean 0.4580613199869792
E -362.2641
AE 2640.7349
MAE 0.27507654825846356
Total pred 5388.5034
Total 6767.041
Mean pred 0.5613024393717448
Mean 0.7049001057942709
E -1378.5376
AE 3792.9546
MAE 0.3950994364420573
Total pred 3826.849
Total 3884.017
Mean pred 0.39863011678059895
Mean 0.4045851135253906
E -57.16765
AE 2905.6436
MAE 0.30267120361328126
Total pred 3907.537
Total 4501.329
Mean pred 0.40703511555989585
Mean 0.4688884480794271
E -593.79224
AE 2630.0896
MAE 0.27396766662597655
Total pred 1079.1554
Total 2081.8518
Mean pred 0.11241202036539713
Mean 0.21685956319173177
E -1002.69666
AE 1789.5858
MAE 0.18641518910725913
Total pred 4817.852
Total 5279.309
Mean pred 0.5018595886230469
Mean 0.5499280293782552
E -461.4571
AE 2712.4363
MAE 0.28254544576009116
Total pred 6558.297
Total 7289.906
Mean pred 0.6831559244791666
Mean 0.7593651835123698
E -731.60864
AE 3505.143
MAE 0.3651190694173177
Total pred 3777.6187
Total 41

Total pred 2247.4897
Total 2407.6538
Mean pred 0.23411351521809895
Mean 0.2507972717285156
E -160.16394
AE 1660.5557
MAE 0.17297454833984374
Total pred 7344.7144
Total 8457.808
Mean pred 0.7650744120279948
Mean 0.8810216267903646
E -1113.0934
AE 4032.1423
MAE 0.42001482645670574
Total pred 3106.7007
Total 3051.464
Mean pred 0.3236146545410156
Mean 0.31786084493001304
E 55.236717
AE 2170.3706
MAE 0.2260802714029948
Total pred 1839.8927
Total 2328.0583
Mean pred 0.1916554896036784
Mean 0.2425060780843099
E -488.16553
AE 1648.2096
MAE 0.1716884994506836
Total pred 2183.631
Total 3056.7961
Mean pred 0.2274615732828776
Mean 0.318416264851888
E -873.1649
AE 2668.4392
MAE 0.27796241760253904
Total pred 2794.1794
Total 2842.646
Mean pred 0.29106035868326824
Mean 0.29610895792643227
E -48.466393
AE 2138.6755
MAE 0.22277870178222656
Total pred 2721.5298
Total 3333.4922
Mean pred 0.283492685953776
Mean 0.34723876953125
E -611.9624
AE 2253.8289
MAE 0.23477383931477866
Total pred 3289.596
Total 384

Total pred 2716.9727
Total 3347.1968
Mean pred 0.28301798502604164
Mean 0.3486663309733073
E -630.22406
AE 2107.4878
MAE 0.2195299784342448
Total pred 2939.5544
Total 2862.8464
Mean pred 0.3062035878499349
Mean 0.29821317036946615
E 76.708
AE 2180.5898
MAE 0.227144775390625
Total pred 4922.169
Total 5147.681
Mean pred 0.5127259318033854
Mean 0.536216786702474
E -225.51227
AE 2895.2583
MAE 0.3015894063313802
Total pred 5399.142
Total 5958.108
Mean pred 0.562410634358724
Mean 0.6206362406412761
E -558.9666
AE 2519.1372
MAE 0.2624101257324219
Total pred 2622.6829
Total 2754.957
Mean pred 0.27319613138834636
Mean 0.2869746907552083
E -132.27428
AE 1778.6564
MAE 0.18527670542399088
Total pred 4017.2935
Total 4486.57
Mean pred 0.4184680684407552
Mean 0.4673510233561198
E -469.27692
AE 3561.9614
MAE 0.3710376485188802
Total pred 2194.329
Total 2726.922
Mean pred 0.22857594807942708
Mean 0.2840543874104818
E -532.59314
AE 1835.7268
MAE 0.19122154235839844
Total pred 5291.8027
Total 5561.5176
M

...

Total pred 4718.678
Total 5201.8804
Mean pred 0.4915289815266927
Mean 0.541862538655599
E -483.20227
AE 2720.0012
MAE 0.2833334604899089
Total pred 3924.9878
Total 4198.7573
Mean pred 0.40885289510091144
Mean 0.4373705546061198
E -273.76944
AE 2299.0437
MAE 0.23948371887207032
Total pred 3985.451
Total 5062.226
Mean pred 0.4151511383056641
Mean 0.5273152160644531
E -1076.775
AE 2921.5151
MAE 0.3043244934082031
Total pred 2084.8381
Total 2040.639
Mean pred 0.21717063903808595
Mean 0.21256656646728517
E 44.19929
AE 1906.7378
MAE 0.19861852010091147
Total pred 2808.5718
Total 2809.1719
Mean pred 0.292559560139974
Mean 0.2926220703125
E -0.5999832
AE 2642.034
MAE 0.2752118682861328
Total pred 6527.267
Total 7747.1934
Mean pred 0.6799236551920573
Mean 0.8069993082682292
E -1219.926
AE 4003.701
MAE 0.41705217997233074
Total pred 2237.4783
Total 3156.4617
Mean pred 0.2330706532796224
Mean 0.328798090616862
E -918.9834
AE 2056.6123
MAE 0.21423044840494793
Total pred 2899.358
Total 3479.476
Mea

Total pred 3283.1284
Total 4345.314
Mean pred 0.34199254353841146
Mean 0.4526368713378906
E -1062.1857
AE 2733.9287
MAE 0.28478424072265623
Total pred 4383.923
Total 4791.834
Mean pred 0.4566586303710938
Mean 0.49914937337239584
E -407.91098
AE 2414.023
MAE 0.2514607238769531
Total pred 3054.1072
Total 3663.1777
Mean pred 0.3181361643473307
Mean 0.38158101399739586
E -609.0701
AE 2112.2407
MAE 0.2200250752766927
Total pred 3050.6377
Total 3906.2183
Mean pred 0.31777475992838544
Mean 0.40689773559570314
E -855.5808
AE 1937.1576
MAE 0.20178724924723307
Total pred 4013.3445
Total 5005.3804
Mean pred 0.4180567169189453
Mean 0.5213937886555989
E -992.036
AE 3076.23
MAE 0.3204406229654948
Total pred 4770.0103
Total 5197.153
Mean pred 0.4968760681152344
Mean 0.5413700866699219
E -427.1428
AE 2972.791
MAE 0.30966573079427084
Total pred 5126.99
Total 6672.491
Mean pred 0.5340614827473958
Mean 0.6950511678059896
E -1545.5009
AE 3561.8496
MAE 0.3710260009765625
Total pred 3060.963
Total 3590.3616

Total pred 1412.0045
Total 2026.9642
Mean pred 0.14708380381266276
Mean 0.21114210764567057
E -614.9596
AE 1991.1215
MAE 0.20740848541259765
Total pred 3843.979
Total 4123.9756
Mean pred 0.4004144795735677
Mean 0.42958079020182294
E -279.99652
AE 3086.7434
MAE 0.32153577168782554
Total pred 3938.935
Total 4272.19
Mean pred 0.41030573527018227
Mean 0.445019785563151
E -333.25497
AE 2330.9917
MAE 0.24281163533528646
Total pred 5456.6206
Total 6068.3804
Mean pred 0.5683979797363281
Mean 0.6321229553222656
E -611.7595
AE 3722.2896
MAE 0.3877384948730469
Total pred 4974.0913
Total 6399.9688
Mean pred 0.518134511311849
Mean 0.6666634114583333
E -1425.8779
AE 3575.372
MAE 0.3724345906575521
Total pred 4746.6562
Total 5345.626
Mean pred 0.494443359375
Mean 0.5568360392252604
E -598.9697
AE 3024.902
MAE 0.3150939687093099
Total pred 3749.4688
Total 4255.6895
Mean pred 0.39056966145833333
Mean 0.4433009847005208
E -506.22046
AE 2538.5852
MAE 0.2644359588623047
Total pred 1469.0105
Total 2092.501

Total pred 3582.727
Total 4394.587
Mean pred 0.3732007344563802
Mean 0.45776947021484377
E -811.8594
AE 3376.8433
MAE 0.35175450642903644
Total pred 3569.6853
Total 4686.528
Mean pred 0.3718422190348307
Mean 0.4881799825032552
E -1116.8429
AE 3034.5767
MAE 0.31610173543294273
Total pred 2492.2388
Total 2798.0002
Mean pred 0.2596082051595052
Mean 0.29145835876464843
E -305.76163
AE 2409.3694
MAE 0.2509759775797526
Total pred 6598.1416
Total 6469.1963
Mean pred 0.6873064168294271
Mean 0.6738746134440105
E 128.94547
AE 3938.994
MAE 0.4103118642171224
Total pred 4130.096
Total 4632.046
Mean pred 0.4302183532714844
Mean 0.4825047810872396
E -501.9494
AE 2430.9136
MAE 0.2532201639811198
Total pred 4532.549
Total 4650.576
Mean pred 0.4721405029296875
Mean 0.48443501790364585
E -118.027794
AE 2519.4146
MAE 0.26243901570638023
Total pred 3745.085
Total 4635.6357
Mean pred 0.3901130167643229
Mean 0.48287872314453123
E -890.5508
AE 2854.844
MAE 0.29737958272298176
Total pred 3178.9177
Total 4194.

Total pred 2089.5576
Total 2887.4797
Mean pred 0.21766225179036458
Mean 0.30077913920084637
E -797.92224
AE 1930.8328
MAE 0.20112841288248698
Total pred 3917.6436
Total 4978.846
Mean pred 0.4080878702799479
Mean 0.5186298116048177
E -1061.2023
AE 3100.6
MAE 0.3229791768391927
Total pred 3336.3877
Total 3508.9053
Mean pred 0.3475403849283854
Mean 0.3655109659830729
E -172.51746
AE 2616.0325
MAE 0.27250338236490884
Total pred 4865.312
Total 5907.5146
Mean pred 0.5068033345540365
Mean 0.6153661092122396
E -1042.2024
AE 3342.9517
MAE 0.34822413126627605
Total pred 3188.389
Total 4158.285
Mean pred 0.33212384541829426
Mean 0.43315470377604165
E -969.8963
AE 2663.7231
MAE 0.27747116088867185
Total pred 5099.589
Total 5474.853
Mean pred 0.5312071736653646
Mean 0.5702971903483073
E -375.26398
AE 2760.9626
MAE 0.2876002756754557
Total pred 1938.0886
Total 2642.0544
Mean pred 0.2018842315673828
Mean 0.27521400451660155
E -703.9659
AE 1884.4045
MAE 0.1962921396891276
Total pred 2892.884
Total 312

In [8]:
y_pred = model.predict(x_test)
print(y_pred.shape)

(10805, 80, 120, 1)


In [10]:
y_test = y_test.flatten()
print(y_test.shape)
y_pred = y_pred.flatten()
print(y_pred.shape)

(103728000,)
(103728000,)


In [11]:
## MEAN ABSOLUTE ERROR

MAE(y_test, y_pred)

0.27195457

In [13]:
##MEAN SQUARE ERROR

mse = MSE(y_test, y_pred)
print(f"MSE: {mse}")

MSE: 0.6010271906852722


In [14]:
## root mean square error

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

RMSE: 0.7752594351768494


### Validando Red VGG16 1000, 700, 500 HGT

In [15]:
model2 = load_model("/opt/datos/entrenamientos/larraondo/vgg16_0_3_5.h5")

In [16]:
x_test = np.load("/opt/datos/dataset/larraondo/x_test-0_3_5_levels.npy")
y_test = np.load("/opt/datos/dataset/larraondo/y_test-0_3_5_levels.npy")

print(x_test.shape)
print(y_test.shape)

(14023, 80, 120, 3)
(14023, 80, 120, 1)


In [17]:
y_pred = model2.predict(x_test)
print(y_pred.shape)

(14023, 80, 120, 1)


In [19]:
y_test = y_test.flatten()
print(y_test.shape)
y_pred = y_pred.flatten()
print(y_pred.shape)

(134620800,)
(134620800,)


In [20]:
## MEAN ABSOLUTE ERROR

MAE(y_test, y_pred)

0.2635415

In [21]:
##MEAN SQUARE ERROR

mse = MSE(y_test, y_pred)
print(f"MSE: {mse}")

MSE: 0.568504810333252


In [22]:
## root mean square error

rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

RMSE: 0.7539925575256348
